In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import numpy as np

# class FullSizeTestDataset(Dataset):
#     def __init__(self, directory):
#         self.directory = directory
#         self.image_files = [os.path.join(directory, f) for f in os.listdir(directory)]
#         self.transform = transforms.Compose([
#             transforms.ToTensor(),
#             # Add any other transformations here
#         ])

#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         img_path = self.image_files[idx]
#         img = Image.open(img_path)
#         img = img.resize((2048, 1536))  # Resize if necessary to handle edge cases
#         patches = self.image_to_patches(img)
#         return patches

#     def image_to_patches(self, img):
#         # Assuming img is a PIL image
#         patches = []
#         for i in range(0, img.width, 100):
#             for j in range(0, img.height, 100):
#                 # Handle edge cases for the last row and column
#                 width, height = min(100, img.width - i), min(100, img.height - j)
#                 patch = img.crop((i, j, i+width, j+height))
#                 patch = self.transform(patch)
#                 patches.append(patch)
#         return torch.stack(patches)  # Return a tensor of all patches


In [5]:
# Load your model
import torch
import numpy as np
from model import ResNeXt
from dataloader import get_dataloaders
from torchvision.utils import save_image
model = ResNeXt().to('cuda')
epoch = 6410

checkpoint = torch.load(f'checkpoints/model_epoch_{epoch}.pth')
model.load_state_dict(checkpoint['generator_state_dict'])

# Prepare DataLoader
batch_size = 50
sample_size = 100
train_loader, test_loader = get_dataloaders(batch_size=batch_size, sample_size=sample_size)
# test_dataset = FullSizeTestDataset('data/iphone/test_data/full_size_test_images')
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Evaluate
model.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        real, target = data
        real = real.to('cuda')
        target = target.to('cuda')
        output = model(real)
        comparison = torch.cat((real[0], output[0]), dim=2)
        save_image(comparison, f'images/output_{i}.jpg')
        if i == 4:
            break
